# Text Generation using LSTM Network
#### ディープラーニングによる文章生成

![](https://isaacchanghau.github.io/img/deeplearning/lstmgru/lstmandgru.png)

## What is GRU Net Work?
  
テキスト生成は一種の言語モデリング問題。  
言語モデリングは、テキスト読み上げ、会話システム、テキスト要約など、多数の自然言語処理タスクの中心的な問題。    
訓練された言語モデルは、テキストで使用されている前の一連の単語に基づいて、単語の出現の可能性を学習する。  
言語モデルは、文字レベル、n-gramレベル、文レベル、または段落レベルでも操作できる。  
このノートでは、最先端のリカレントニューラルネットワークを実装してトレーニングすることによって、自然言語テキストを生成するための言語モデルを作成する方法について説明していく
。

## 今回はニュースの本文からタイトルを自動生成します  
  
#### Process  
1. データの準備  
2. 文章のお掃除（記号削除、小文字統一）  
3. 単語に切り分ける  
4. トークン化＝数値化  
5. パディングで変数の長さを統一  
6. LSTMの実装

## 1. Import the libraries ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np

## 2. Load the dataset データのロード

In [2]:
import os
path = os.getcwd()
print(path)

/Users/akr712/Desktop/今日やること/文章生成


In [4]:
headlines = []
for filename in os.listdir(path):
    if "Articles" in filename:
        article_df = pd.read_csv(path + "/" + filename)
        headlines.extend(list(article_df["headline"].values))
        break
        
headlines = [ h for h in headlines if h != "Unknown" ]
print("The number of headline is :", len(headlines))

The number of headline is : 829


In [ ]:
headlines

## 3. Dataset preparation 前処理

### 3.1 Dataset cleaning  データクリーニング
  
記号を取り除き、「文字」と「数字」だけ残す。また小文字で統一する。

In [23]:
import string
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_text(headline):
    text = "".join( word for word in headline if word not in string.punctuation ).lower()
    text = text.encode("utf8").decode("ascii", "ignore")
    return text

# 元データのタイトルに含まれる単語群から独自のコーパスを作成
corpus = [ clean_text(headline) for headline in headlines ]

In [ ]:
corpus

### 3.2 Generating Sequence of N-gram Tokens 文章の単語化&数値化

- 自然言語処理では、テキストを単語単位に分解してベクトル化するのが主流である。  
- N-gram は、Morphological Analysis（形態要素解析）に並ぶ代表的な単語の切り出し手法のひとつ。  
- 具体的には、N-gramとは自然言語（テキスト）を連続するN個の文字、もしくはN個の単語単位で切り出す手法のこと。  
- 強みは「コーパス」が事前に入らないこと、弱みは切り出した単語数が肥大化しやすい点。  
- ex:) "I voted for Trump." n=2 => "I voted", "for Trump"

In [26]:
vocab = []
for line in corpus:
    words = line.split()
    for word in words:
        vocab.append(word)

vocabraly = set(vocab)

In [27]:
len(vocabraly)

2287

In [116]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(2000)
tokenizer.fit_on_texts(corpus)
word2index = tokenizer.word_index
len(word2index)

2287

In [ ]:
word2index

In [118]:
dictionary = {}
rev_dictionary = {}
for word, idx in word2index.items():
    if idx > 1406:
        continue
    dictionary[word] = idx
    rev_dictionary[idx] = word

In [119]:
max(rev_dictionary.keys())

1406

In [120]:
input_seqences = tokenizer.texts_to_sequences(corpus)

In [ ]:
input_seqences

In [122]:
len(input_seqences)

829

### 3.3 Padding the Sequences and obtain Variables
#### パディングによって固定長データを作り、説明変数を得る

In [123]:
input_data = []
target = []
for line in input_seqences:
    for i in range(1, len(line)-1):
        input_data.append(line[:i])
        target.append(line[i+1])

In [124]:
input_data[:5]

[[660],
 [660, 117],
 [660, 117, 72],
 [660, 117, 72, 73],
 [660, 117, 72, 73, 661]]

In [125]:
target[:5]

[72, 73, 661, 662, 63]

In [126]:
MAX_LEN = 0
for seq in input_data:
    if len(seq) > MAX_LEN:
        MAX_LEN = len(seq)
MAX_LEN

15

In [127]:
input_data = pad_sequences(input_data, maxlen=MAX_LEN, padding="post", truncating="post")

In [128]:
len(input_data[0])

15

In [129]:
input_data.shape

(3455, 15)

In [130]:
target = to_categorical(target, num_classes=total_words)

In [131]:
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [132]:
target.shape

(3455, 2288)

In [135]:
VOCAB_SIZE = 2001
VOCAB_SIZE

2001

In [151]:
MAX_LEN

15

## 4. LSTMs for Text Generation 長短期記憶層アルゴリズムの実装

### 4.1 LSTM ( Long Short-Term Memory  )    
  
1. Input Layer : Takes the sequence of words as input  
2. LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.  
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer.  
4. Output Layer : Computes the probability of the best possible next word as output  

![title](https://cdn-images-1.medium.com/max/1600/1*yBXV9o5q7L_CvY7quJt3WQ.png)

In [143]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping

In [138]:
model = Sequential()

"""入力層"""
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))

"""隠れ層"""
model.add(LSTM(units=100))
model.add(Dropout(rate=0.1))

"""出力層 活性化関数は多層のソフトマックス関数"""
model.add(Dense(units=target.shape[1], activation="softmax"))

In [139]:
"""損失関数と最適化手法の設定"""
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [140]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 15, 100)           200100    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 2288)              231088    
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________


#### LSTMモデルにテストデータを学習させていく！

In [141]:
"""予測ラベルと正解ラベルを用意する"""
model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
3455/3455 [==============================] - 5s 1ms/step - loss: 7.2371
Epoch 2/5
3455/3455 [==============================] - 3s 940us/step - loss: 6.6598
Epoch 3/5
3455/3455 [==============================] - 3s 988us/step - loss: 6.5953
Epoch 4/5
3455/3455 [==============================] - 3s 932us/step - loss: 6.5657
Epoch 5/5
3455/3455 [==============================] - 3s 991us/step - loss: 6.5435


### 4.2 GRU ( Gated recurrent unit )

In [144]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))
gru_model.add(GRU(units=100))
gru_model.add(Dropout(rate=0.1))
gru_model.add(Dense(units=target.shape[1], activation="softmax"))

In [145]:
gru_model.compile(loss="categorical_crossentropy", optimizer="adam")

In [146]:
gru_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 15, 100)           200100    
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2288)              231088    
Total params: 491,488
Trainable params: 491,488
Non-trainable params: 0
_________________________________________________________________


#### GRUモデルにテストデータを学習させていく！

In [147]:
gru_model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
3455/3455 [==============================] - 5s 1ms/step - loss: 7.2592
Epoch 2/5
3455/3455 [==============================] - 3s 882us/step - loss: 6.6681
Epoch 3/5
3455/3455 [==============================] - 3s 889us/step - loss: 6.5904
Epoch 4/5
3455/3455 [==============================] - 3s 885us/step - loss: 6.5545
Epoch 5/5
3455/3455 [==============================] - 3s 922us/step - loss: 6.5160


## 5. Generating the text タイトルにふさわしいテキストを自動生成する

In [142]:
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

In [152]:
def text_generater(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding="post")
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [157]:
text1 = "Trump decided"
text_generater(text1, 5, model, MAX_LEN)

'Trump Decided The The The The The'

In [158]:
text_generater(text1, 5, gru_model, MAX_LEN)

'Trump Decided The The The The The'

## I need more data to training I guess.... 学習不足